<a href="https://colab.research.google.com/github/rprimi/colB5BERT/blob/main/python/b5_contextualreps_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Getting BERT embeddings and calculating consine similarity between items and posts tokens
Ricardo Primi
Projeto Final, UNICAMP, Disciplina IA368 Deep Learning aplicada a buscas

### General set-up



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
!ls -lh


total 28K
drwxr-xr-x 7 root root 4.0K Jun 22 14:09 colB5BERT
-rw-r--r-- 1 root root  282 Jun 22 14:08 colB5BERT.Rproj
drwxr-xr-x 2 root root 4.0K Jun 22 14:08 data
drwxr-xr-x 2 root root 4.0K Jun 22 14:08 docs
drwxr-xr-x 2 root root 4.0K Jun 22 14:08 python
drwxr-xr-x 2 root root 4.0K Jun 22 14:08 R
-rw-r--r-- 1 root root  103 Jun 22 14:08 README.md


In [ ]:
!pip3 install transformers

In [23]:
!git clone https://github.com/rprimi/colB5BERT.git

%cd /content/colB5BERT
!git pull



Cloning into 'colB5BERT'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (198/198), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 198 (delta 121), reused 56 (delta 22), pack-reused 0
Receiving objects: 100% (198/198), 13.97 MiB | 21.91 MiB/s, done.
Resolving deltas: 100% (121/121), done.
/content/colB5BERT
Already up to date.


In [21]:
import os
import pandas as pd
import numpy as np
import textwrap
import pickle
import h5py
import logging

from transformers import BertModel, BertTokenizer
from transformers import RobertaModel, RobertaTokenizer

import torch
from torch.nn.functional import cosine_similarity
from tqdm import tqdm


Modules `vsm`, `utils` and `sst` are from Stanford's CS224u https://github.com/cgpotts/cs224u

In [ ]:
import sys
sys.path.append('/content/colB5BERT/python/')

import utils
import vsm
import sst


In [27]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: nvidia-smi: command not found


### Data

In [28]:
b5_data = pd.read_csv('/content/colB5BERT/data/db_textos.splitted.csv', sep=';')
b5_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11537 entries, 0 to 11536
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              11537 non-null  int64 
 1   id_divisao      11537 non-null  int64 
 2   texto_dividido  11537 non-null  object
dtypes: int64(2), object(1)
memory usage: 270.5+ KB


In [29]:
b5_data

,id,id_divisao,texto_dividido
0,100,1,ajudando porque a zuzu é um amor e tem a voz f...
1,100,2,vai ter share sim e se reclamar dou share mais...
2,100,3,"quanto parece , A , $NUMBER$ . MvC $NUMBER$ CL..."
3,100,4,$NUMBER$ jeitos de dar entry então é só sucess...
4,100,5,quiser se vira Esse livro é de co-autoria de $...
...,...,...,...
11532,999,6,"amo muito ! < $NUMBER$ < $NUMBER$ "" Fique por ..."
11533,999,7,pai ! Feliz aniversário ! < $NUMBER$ < $NUMBER...
11534,999,8,rei do $NAME$ Club de $NAME$ Oeste : $NAME$ $N...
11535,999,9,todo tipo de público . A realização do projeto...


In [8]:
import pandas as pd
base_itens_b5 = pd.read_excel('/content/colB5BERT/data/base_itens.xlsx')


base_itens_b5
base_itens_b5.info()
# base_itens_b5['item_pt_text'].tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ord0_index    415 non-null    int64  
 1   test          415 non-null    object 
 2   coditem       415 non-null    object 
 3   item_pt_text  415 non-null    object 
 4   item_en_text  415 non-null    object 
 5   domain        413 non-null    object 
 6   facet         413 non-null    object 
 7   pole          415 non-null    int64  
 8   seman_pairs   273 non-null    float64
dtypes: float64(1), int64(2), object(6)
memory usage: 29.3+ KB


### Loading Transformer models
Specify a model, a tokenizer, and load a model pretrained weights:

In [10]:
bert_weights_name = 'neuralmind/bert-base-portuguese-cased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_weights_name)
bert_model = BertModel.from_pretrained(bert_weights_name)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### The basics of tokenizing


In [ ]:
def print_cell(df, row, column, wrap_length=80):
    if row < len(df) and column in df.columns:
        text = df.loc[row, column]
        print('\n'.join(textwrap.wrap(text, width=wrap_length)))
    else:
        print("Invalid row or column")

print_cell(b5_data, 3, 'texto_dividido')

ex_ids = bert_tokenizer.encode(b5_data.loc[3, 'texto_dividido'], add_special_tokens=True)
bert_tokenizer.convert_ids_to_tokens(ex_ids)


### Getting BERT embeddings

To obtain the representations for a batch of examples, we use the `forward` method of the model, as follows:

In [ ]:
with torch.no_grad():
    reps = bert_model(torch.tensor([ex_ids]), output_hidden_states=True)

In [11]:
def tokenize_texts(texts):
    # Tokenize each text and convert to input IDs
    input_ids = [bert_tokenizer.encode(text, add_special_tokens=True) for text in texts]
    return input_ids


def tokenize_texts(bert_tokenizer, texts):
    tokenized_texts = []
    for text in texts:
        encoded_text = bert_tokenizer.encode(text, add_special_tokens=True)
        # truncate the encoded text to the first 512 tokens
        encoded_text = encoded_text[:512]
        # encoded_text = encoded_text
        tokenized_texts.append(encoded_text)
    return tokenized_texts



In [ ]:
# tokenize_texts(ex_of_texts)
ex_of_texts = b5_data.iloc[[0, 1, 2, 3], b5_data.columns.get_loc('texto_dividido')].tolist()
lengths = [len(sublist) for sublist in tokenize_texts(bert_tokenizer, ex_of_texts)]

print(lengths)  # Output: [3, 2, 4]

[512, 512, 477, 512]


In [ ]:
# Many tokens each item has?
lengths_items = [len(sublist) for sublist in tokenize_texts(bert_tokenizer, base_itens_b5['item_pt_text'].tolist())]



In [11]:
def get_bert_embeddings(bert_model, examples, layers):
    # Move model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    bert_model = bert_model.to(device)

    embeddings = {layer: [] for layer in layers}
    for ex_ids in examples:
        # Convert data to tensor and move to GPU
        ex_ids_tensor = torch.tensor([ex_ids]).to(device)
        with torch.no_grad():
            # Output includes 'last_hidden_state', 'pooler_output', 'hidden_states'
            output = bert_model(ex_ids_tensor, output_hidden_states=True)
            hidden_states = output.hidden_states
            for layer in layers:
                # Verify layer index is valid
                if layer < 0 or layer >= len(hidden_states):
                    print(f"Invalid layer {layer}")
                else:
                    # Hidden states is a tuple. Indexing into it gives a tensor of shape
                    # (batch_size, sequence_length, hidden_size). Since batch_size is 1,
                    # we remove the batch dimension.
                    layer_output = hidden_states[layer].squeeze(0)
                    # Convert back to CPU for further processing or storage
                    embeddings[layer].append(layer_output.to('cpu'))
    return embeddings

In [ ]:
layers = [6, 9, 11, 12]  # Specify the layers you want
layers = [6]
embeddings = get_bert_embeddings(bert_model, tokenize_texts(bert_tokenizer, ex_of_texts), layers)

import pprint
pprint.pprint(embeddings )


dir(embeddings)
vars(embeddings)
import inspect
inspect.getmembers(embeddings)

pprint.pprint(embeddings[9])
len(embeddings[9])
len(embeddings[9][2])
pprint.pprint(embeddings[9][2])
x = embeddings[9][2]
x.shape

dimensions = [len(inner_list) for inner_list in embeddings[9][0]]



### Finally getting the embeddings

In [13]:
# Specify the layers you want
layers = [6, 9, 11, 12]
layers = [6]

layers = [11]

len(b5_data['texto_dividido'].tolist())

embeddings_posts = get_bert_embeddings(bert_model, tokenize_texts(bert_tokenizer, b5_data['texto_dividido'].tolist()), layers)
embeddings_itens = get_bert_embeddings(bert_model, tokenize_texts(bert_tokenizer, base_itens_b5['item_pt_text'].tolist()), layers)



### Save embeddings
This doesn't work. Post embeedings is 56Gb

In [ ]:
def save_embeddings_to_disk(embeddings, filename):
    # Convert tensors to numpy arrays and store them in the same structure
    numpy_embeddings = {str(layer): [t.numpy() for t in tensors] for layer, tensors in embeddings.items()}

    # Use numpy's savez function to store the dictionary
    # We use ** to unpack the dictionary into keyword arguments
    np.savez(filename, **numpy_embeddings)

def load_embeddings_from_disk(filename):
    with np.load(filename) as data:
        embeddings = {layer: data[layer] for layer in data.files}
    return embeddings

filename="/content/drive/MyDrive/colB5BERT/embeddings_itens"

save_embeddings_to_disk(embeddings=embeddings_itens, filename=filename)

filename="/content/drive/MyDrive/colB5BERT/embeddings_posts"
save_embeddings_to_disk(embeddings=embeddings_posts, filename=filename)


/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py:713: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  val = np.asanyarray(val)


Embeddings is a dict of `layers` keys. Each component of one key is composed of `batch size` elments of a tensor with size `num_of_tokens X embedding_dim`. The final structure is `layers X batch size X num_of_tokens X embedding_dim`

In [ ]:
# how many posts
len(embeddings_posts[6])
embeddings_posts[6]

# what is the first element ? of the first post
type(embeddings_posts[6][0])

# what is the dimension
embeddings_posts[6][3].shape


# how many posts
len(embeddings_itens[6])

# what is the first element ? of the first post
type(embeddings_itens[6][0])

# what is the dimension
embeddings_itens[6][0].shape

type(embeddings_itens[6])

torch.Size([512, 768])

### Cosine Similarity

Testing the function

In [ ]:
def calculate_cosine_similarity0(list_A, list_B):
    result = []
    for tensor_A in list_A:
        for tensor_B in list_B:
            # Expand dimensions so that shapes are [n_tokens_A, 1, 768] and [1, n_tokens_B, 768]
            tensor_A_exp = tensor_A.unsqueeze(1)
            tensor_B_exp = tensor_B.unsqueeze(0)
            # Compute cosine similarity
            similarity = cosine_similarity(tensor_A_exp, tensor_B_exp, dim=-1)
            result.append(similarity)
    return result


We can then use cosine_similarity(tensor_A_exp, tensor_B_exp, dim=-1) to compute the cosine similarity between all pairs of tokens from tensor_A and tensor_B, resulting in a tensor of shape [n_tokens_A, n_tokens_B]. Each element (i,j) in the resulting tensor represents the cosine similarity between the ith token from tensor_A and the jth token from tensor_B.



In [ ]:
temp = calculate_cosine_similarity0(embeddings_itens[6][0:10], embeddings_posts[6][0:5])

temp.shape
len(temp) # 10 itens X 5 posts
len(temp[5][2]) # Item 1 com 13 tokens
len(temp[0][9])
temp[0].shape
temp2 = torch.cat(temp.unsqueeze(0), dim=-1)

This code tryes to recover all cosine similarities. But explode RAM

In [ ]:


# user = os.getenv('DB_USER')
# password = os.getenv('DB_PASS')
!pip install PyMySQL
import pymysql

# Establish the connection
conn = pymysql.connect(
    host = os.getenv('LABAPE6_HOST'),
    user = os.getenv('LABAPE6_USER'),
    password = os.getenv('LABAPE6_PASSWORD'),
    db = os.getenv('LABAPE6_DBNAME')
)

# Create a cursor
cur = conn.cursor()

# Execute a query to fetch data
cur.execute("SELECT * FROM your_table")

# Fetch the results
results = cur.fetchall()

for row in results:
    print(row)

# Now, let's insert data into a table
cur.execute("INSERT INTO your_table (col1, col2) VALUES ('data1', 'data2')")

# Commit your changes
conn.commit()

# Close the connection
conn.close()



In [14]:
logging.basicConfig(level=logging.INFO)  # you can change the level to DEBUG if you want more detailed logs
logger = logging.getLogger(__name__)


def calculate_cosine_similarity1(list_A, list_B, batch_size=1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    result = []

    # Move list_A tensors to GPU
    list_A = [tensor.to(device) for tensor in list_A]

    # Process batch_size tensor_A at a time
    for i in tqdm(range(0, len(list_A), batch_size), desc='Processing', dynamic_ncols=True):
        batch_A = list_A[i:i+batch_size]
        batch_A = torch.nn.utils.rnn.pad_sequence(batch_A, batch_first=True).unsqueeze(2)  # [batch_size, max_n_tokens_A, 1, 768]
        similarities = []

        for tensor_B in list_B:
            tensor_B = tensor_B.to(device).unsqueeze(0).unsqueeze(0)  # [1, 1, n_tokens_B, 768]
            similarity = torch.nn.functional.cosine_similarity(batch_A, tensor_B, dim=-1)  # [batch_size, max_n_tokens_A, n_tokens_B]
            similarity = similarity.half()  # Reduce precision to float16
            similarities.append(similarity.cpu().numpy())  # Move similarity to CPU and convert to NumPy array
            del tensor_B  # Delete tensor_B from GPU memory

            # Store the batch's similarities to the result
            result.append(similarities)

    return result


In [15]:
cosim_L11 = calculate_cosine_similarity1(embeddings_posts[11], embeddings_itens[11], batch_size=1)


import h5py

def save_to_hdf5(data, filename, dataset_name):
    with h5py.File(filename, 'w') as hf:
        hf.create_dataset(dataset_name, data=data)


filname="/content/drive/MyDrive/colB5BERT/cosim_L11.hd5"

save_to_hdf5(cosim_L11, filname, 'cosim_data')





Processing: 100%|██████████| 11537/11537 [19:22<00:00,  9.93it/s]


In [17]:

import numpy as np
import pickle

# Determine the size of each chunk
CHUNK_SIZE = 11 * 11537  # This is an example, adjust based on your data and system capacity

# Load your large data
#with open('/content/drive/MyDrive/colB5BERT/cosim_L11.pkl', 'rb') as f:
#    cosim_L11 = pickle.load(f)

# Determine the number of chunks
num_elements = len(cosim_L11)
NUM_CHUNKS = num_elements // CHUNK_SIZE
if num_elements % CHUNK_SIZE:
    NUM_CHUNKS += 1

# Save each chunk to a separate file
for i in range(NUM_CHUNKS):
    start = i * CHUNK_SIZE
    end = min((i + 1) * CHUNK_SIZE, num_elements)
    chunk = cosim_L11[start:end]

    with open(f'/content/drive/MyDrive/colB5BERT/cosim_L11_chunk_{i}.pkl', 'wb') as f:
        pickle.dump(chunk, f)


38

This code saves the Top K cosine similarities with text divided tokens with all tokens in all items

In [ ]:
def calculate_cosine_similarity2(list_A, list_B, topk=5, batch_size=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    result = []

    # Move list_A tensors to GPU
    list_A = [tensor.to(device) for tensor in list_A]

    # Process batch_size tensor_A at a time
    for i in tqdm(range(0, len(list_A), batch_size), desc='Processing', dynamic_ncols=True):
        batch_A = list_A[i:i+batch_size]
        batch_A = torch.nn.utils.rnn.pad_sequence(batch_A, batch_first=True).unsqueeze(2)  # [batch_size, max_n_tokens_A, 1, 768]

        for tensor_B in list_B:
            tensor_B = tensor_B.to(device).unsqueeze(0).unsqueeze(0)  # [1, 1, n_tokens_B, 768]
            similarity = torch.nn.functional.cosine_similarity(batch_A, tensor_B, dim=-1)  # [batch_size, max_n_tokens_A, n_tokens_B]

             # [batch_size, max_n_tokens_A, n_tokens_B]

            current_topk = min(topk, similarity.size(-1))  # Ensure topk isn't larger than the number of tokens in tensor_B

            topk_values, topk_indices = torch.topk(similarity, current_topk, dim=-1)  # [batch_size, max_n_tokens_A, current_topk]
            result.append((topk_values.cpu().numpy(), topk_indices.cpu().numpy()))  # Move topk_values and topk_indices to CPU
            del tensor_B  # Delete tensor_B from GPU memory

    return result


Trying to get embeddings in batched slices

In [ ]:
def calculate_cosine_similarity3(list_A, posts=b5_data['texto_dividido'].tolist(), topk=5, batch_size=25, layer=6):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    result = []


    # Move list_A tensors to GPU
    list_A = [tensor.to(device) for tensor in list_A]

    # Process batch_size tensor_A at a time
    for i in tqdm(range(0, len(posts), batch_size), desc='Processing', dynamic_ncols=True):
        # batch_A = list_A[i:i+batch_size]
        list_A = torch.nn.utils.rnn.pad_sequence(list_A, batch_first=True).unsqueeze(2)  # [batch_size, max_n_tokens_A, 1, 768]


        batch_posts = get_bert_embeddings(bert_model, tokenize_texts(bert_tokenizer, posts[i:i+batch_size]), layers=[layer])

        for tensor_B in batch_posts[layer]:
            tensor_B = tensor_B.to(device).unsqueeze(0).unsqueeze(0)  # [1, 1, n_tokens_B, 768]
            similarity = torch.nn.functional.cosine_similarity(list_A, tensor_B, dim=-1)  # [batch_size, max_n_tokens_A, n_tokens_B]

            current_topk = min(topk, similarity.size(-1))  # Ensure topk isn't larger than the number of tokens in tensor_B

            topk_values, topk_indices = torch.topk(similarity, current_topk, dim=-1)  # [batch_size, max_n_tokens_A, current_topk]
            result.append((topk_values.cpu(), topk_indices.cpu()))  # Move topk_values and topk_indices to CPU
            del tensor_B  # Delete tensor_B from GPU memory

    return result



The `result` is a list of tuples, where each tuple consists of `topk_values` and `topk_indices`.

1. `topk_values` is a PyTorch tensor of shape `[batch_size, max_n_tokens_A, current_topk]`, where `batch_size` is the number of `tensor_A` processed at a time, `max_n_tokens_A` is the maximum number of tokens in `tensor_A` in the current batch, and `current_topk` is the number of top similarity scores to return, which is either the `topk` parameter or the number of tokens in `tensor_B`, whichever is smaller. This tensor holds the `current_topk` highest cosine similarity scores for each token in each `tensor_A` in the batch against `tensor_B`.

2. `topk_indices` is a PyTorch tensor of the same shape as `topk_values` (`[batch_size, max_n_tokens_A, current_topk]`). This tensor holds the indices of the `current_topk` highest cosine similarity scores in `tensor_B`.

So, in `result`, you will have a list of tuples (each for one `tensor_B`) and each tuple contains two tensors (`topk_values` and `topk_indices`) of shape `[batch_size, max_n_tokens_A, current_topk]`. As the function processes `list_A` in batches, the exact length of `result` will be `len(list_A) / batch_size * len(list_B)` (if `len(list_A)` is exactly divisible by `batch_size`), or `len(list_A) // batch_size * len(list_B) + len(list_B)` (if it's not).

In [ ]:
cosim_top5_L6 = calculate_cosine_similarity2(embeddings_itens[6][0:3], embeddings_posts[6][0:10], topk = 5, batch_size=2)


In [ ]:
len(cosim_top5_L6)

embeddings_itens[6][1].shape
cosim_top5_L6[0][0].shape


len(embeddings_L6)
batch_size = 5
len(embeddings_L6) == len(list(range(0, len(embeddings_itens[6]), batch_size))) * len(embeddings_posts[6])

len(embeddings_L6[0])
len(embeddings_L6[0][:])
len()


[ print(x) for x in embeddings_L6[0][0] ]

temp = embeddings_L6[0][0]
len(temp[0][412])
temp1 = temp[512]
temp1[412]

temp1 = temp[412][412]

len(temp1[5])
for i in range(2):  # Change this number to see more/less
    topk_values, topk_indices = embeddings_L6[i]
    print("TopK Values Tensor:")
    print(topk_values)
    print("TopK Indices Tensor:")
    print(topk_indices)
    print("\n---\n")


In [ ]:
# temp = calculate_cosine_similarity(embeddings_itens[6], embeddings_posts[6])
len(embeddings_itens[6][0])
len(embeddings_itens[6][1])

len(embeddings_posts[6][100])


len(temp[511][0])
import numpy as np
temp[511].numpy()

In [ ]:
import pickle

with open('/content/drive/MyDrive/colB5BERT/embeddings_L6.pkl', 'wb') as f:
    pickle.dump(embeddings_L6, f)

In [ ]:
import numpy as np

# Save each tuple of (topk_values, topk_indices) in the result
for i, (topk_values, topk_indices) in enumerate(embeddings_L6):
    np.savez(f'/content/drive/MyDrive/colB5BERT/output_{i}.npz', values=topk_values, indices=topk_indices)

data = np.load('output_0.npz')
values = data['values']
indices = data['indices']


In [ ]:
import numpy as np
import pandas as pd

def save_to_csv(tensor_list, filename):
    flattened_tensors = [tensor.numpy().flatten() for tensor in tensor_list]
    shapes = [tensor.shape for tensor in tensor_list]

    data_df = pd.DataFrame({
        'tensor': [tensor.tolist() for tensor in flattened_tensors],
        'shape': shapes
    })

    data_df.to_csv(filename, index=False)

# Assuming your list of tensors is called tensor_list
save_to_csv(embeddings_itens[6], 'embeddings_itens6.csv')



In [10]:


cosims_L6 = [y.numpy() for y in [x[0] for x in data]]

# Save the result as a new pickle file
with open("/content/drive/MyDrive/colB5BERT/cosim_L6.pkl", "wb") as file:
    pickle.dump(cosims_L6, file)

ids_cosims_L6 = [y.numpy() for y in [x[1] for x in data]]

ids_cosims_L6[0:2]

len(cosims_L6)

with open("/content/drive/MyDrive/colB5BERT/ids_cosims_L6.pkl", "wb") as file:
    pickle.dump(ids_cosims_L6, file)


with open("/content/drive/MyDrive/colB5BERT/cosim_L6.pkl", "wb") as file:
    pickle.dump(cosims_L6, file)

with open("/content/drive/MyDrive/colB5BERT/cosim_L6_full.pkl", "wb") as file:
    pickle.dump(embeddings_L6, file)


### Restructuring data

In [14]:
# Load the pickle file
with open("/content/drive/MyDrive/colB5BERT/cosim_L6.pkl", "rb") as file:
    data = pickle.load(file)


type(data)
len(data)
len(data[0])
data[0].shape
type(data[0])
data[0][0].shape

array([[0.9999438 , 0.28923154, 0.16323651, 0.15610822, 0.15447576],
       [0.54535484, 0.5104598 , 0.50956935, 0.5010768 , 0.49993923],
       [0.6487386 , 0.6451509 , 0.64438605, 0.5922517 , 0.58099186],
       [0.5901557 , 0.58609843, 0.5840987 , 0.5761634 , 0.5509137 ],
       [0.48015648, 0.4667238 , 0.46271023, 0.45601392, 0.45178825],
       [0.5916519 , 0.56757987, 0.55520004, 0.5527642 , 0.5513735 ],
       [0.65486205, 0.65253234, 0.6455029 , 0.64238495, 0.6225333 ],
       [0.49928924, 0.49720046, 0.47456792, 0.46936598, 0.46704215],
       [0.70637774, 0.656662  , 0.60018235, 0.5642161 , 0.5619816 ],
       [0.46530312, 0.4589913 , 0.45810091, 0.45756498, 0.45258853],
       [0.6122791 , 0.60375166, 0.5908047 , 0.5653817 , 0.5639473 ],
       [0.44506344, 0.44214535, 0.44137588, 0.43476218, 0.43126088],
       [0.56115127, 0.5407094 , 0.5342648 , 0.5315968 , 0.527797  ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)

- data isa  list of numpy.ndarray of shape (b, tk, r). r is a cosine similarity.
- this code first sum along cosine similarities r reducing to (b, tk, 1)
- then average along the token dimesnion having shape (b, 1)?




In [9]:
import numpy as np

# Initializing an empty list to hold the final results
final_data = []

# Loop through each numpy.ndarray in your data_list
for dat in data:

    # Sum along the r-axis, resulting in an array of shape (b, tk)
    dat_sum = np.sum(dat, axis=-1)

    # Then take the mean along the tk-axis, resulting in an array of shape (b,)
    dat_mean = np.mean(dat_sum, axis=-1)

    # Append the resulting array to your final_data list
    final_data.append(dat_mean)

# Convert final_data to numpy array
final_data = np.array(final_data)

# Now final_data is an numpy.ndarray of shape (b, )

final_data.shape

(957571, 5)

In [ ]:
import numpy as np

# Let's say final_data is your list of arrays
final_data_ex = [np.random.rand(3, 5) for _ in range(6)]  # This just creates some example data

# Using vstack
stacked_data = np.vstack(final_data_ex)

len(stacked_data)

# Using concatenate
stacked_data = np.concatenate(final_data_ex, axis=0)

# Now stacked_data is a single array with shape (10n, 5), assuming final_data had 10 elements

print(final_data_ex)
print(stacked_data )

In [10]:
final_data = np.concatenate(final_data, axis=0)
final_data.shape




(4787855,)

In [12]:
with open("/content/drive/MyDrive/colB5BERT/cosim_scores_L6.pkl", "wb") as file:
    pickle.dump(final_data, file)